Goal for 6/25
    - add logic for checking year
    - add logic for comparing education
    - add logic to parse work summary

In [2]:
from selenium import webdriver
import csv
import time
import random

In [88]:
search_list = []
#input = pd.read_csv('ten_sample.csv')
with open('hundo_sample.csv', 'r') as input:
    for row in input:
        search_list.append(row)
search_list.pop(0)        

'"First\tLast\tClass"\n'

In [3]:
driver = webdriver.Chrome()

In [4]:
url = 'https://www.linkedin.com/'
username = 'hbb3xe@virginia.edu'
password = 'S+zMKr5o'

In [5]:
driver.get(url)

In [91]:
driver.find_element_by_id('login-email').send_keys(username)
driver.find_element_by_id('login-password').send_keys(password)

In [92]:
driver.find_element_by_name('submit').click()

In [93]:
driver.find_element_by_id('advanced-search').click()

In [77]:
school = 'University of Virginia'
school2 = 'McIntire'

In [94]:
scraped_info = []
for row in search_list:
    first, last, year = row.split('\t')
    first_name = first.replace('"', '')
    last_name = last.strip("\"\n")
    year = year.replace('"\n', '')
    print("Processing: {} {} {}".format(first, last, year))
    driver.find_element_by_id('advanced-search').click()
    time.sleep(random.gauss(1.5, .5))
    
    driver.find_element_by_id('advs-firstName').clear()
    driver.find_element_by_id('advs-lastName').clear()
    driver.find_element_by_id('advs-school').clear()
    
    driver.find_element_by_id('advs-firstName').send_keys(first_name)
    driver.find_element_by_id('advs-lastName').send_keys(last_name)
    driver.find_element_by_id('advs-school').send_keys(school)
    driver.find_element_by_class_name('submit-advs').click()
    time.sleep(random.gauss(4.2, .5))
    
    num_results = driver.find_element_by_class_name('search-info').text.split('\n')[0]
    num_results = num_results.strip(" results")
    #num_results = int(num_results)
    if num_results == '0':
        info = 'Could not scrape' + first_name + ' ' + last_name
        scraped_info.append([info])
        continue
    
    results_summary = driver.find_element_by_id('results-container').text.split('\n')
    #print('results summary: {}'.format(results_summary))
    scraped_name = results_summary[0].replace('1st|2nd|3rd|, CPA','').replace('Premium Badge','')
    #print('scraped name: {}'.format(scraped_name))
    summary_position = results_summary[1]
    
    full_name = first_name + " " + last_name
    name_match = "True" if scraped_name == full_name else "False"
    
    driver.find_element_by_css_selector('a.title.main-headline').click()
    time.sleep(random.gauss(1.5, .5))
    
    education_text = driver.find_element_by_id('background-education').text.lstrip('Education\n')
    school_match = "True" if school in education_text else "False"
    year_match = "True" if year in education_text else "False"
    parsed_education_text = education_text.split('\n')
    
    work_experience = driver.find_element_by_id('background-experience-container').text.lstrip('Experience\n')
    parsed_work_experience = work_experience.split('\n')
    current_position = parsed_work_experience[0]
    current_company = parsed_work_experience[1]
    current_location = parsed_work_experience[2].rsplit(')')[1]
    
    if name_match == 'False' and school_match == 'False' and year_match == 'False':
        try_mcintyre(driver=driver, first_name, last_name, school=school2)
    
    
    driver.find_element_by_id('advanced-search').click()
    time.sleep(random.gauss(1.5, .5))
    
    info = (first_name, last_name, year, num_results, scraped_name, summary_position,
            current_position, current_company, current_location,
            parsed_education_text, parsed_work_experience, name_match, school_match, year_match)
    scraped_info.append(info)
    
    #if name_match == 'False' and school_match == 'False' and year_match == 'False':
    #    driver.find_element_by_id('advanced-search').click()
    #    time.sleep(random.gauss(1.5, .5))
    
    #    driver.find_element_by_id('advs-firstName').clear()
    #    driver.find_element_by_id('advs-lastName').clear()
    #    driver.find_element_by_id('advs-school').clear()
    
    #    driver.find_element_by_id('advs-firstName').send_keys(first_name)
    #    driver.find_element_by_id('advs-lastName').send_keys(last_name)
    #    driver.find_element_by_id('advs-school').send_keys(school2)
    #    driver.find_element_by_class_name('submit-advs').click()
    #    time.sleep(random.gauss(4.2, .5))
        

Processing: "Sutton White 2016
Processing: "Kevin Hsu 2014
Processing: "Ryan Ho 2014
Processing: "Devon Holland 2014
Processing: "Ajay Sundar 2014
Processing: "Graham Campbell 2014
Processing: "David Plon 2011
Processing: "Peter Lizza 2014
Processing: "Alex Sullivan 2014
Processing: "Stephen West 2014
Processing: "Rachel Harvey 2012
Processing: "Kyle Bye 2014
Processing: "Peter Restrepo 2014
Processing: "Brian Ohlhausen 2014
Processing: "Scott Lucas 2014
Processing: "Nicholas Jones 2014
Processing: "John Tubesing 2014
Processing: "William Cozean 2011
Processing: "Julian El-Abidin 2013
Processing: "Eugene Chang 2014
Processing: "Christian Pinto 2017
Processing: "Connor Davis 2015
Processing: "Lex Alvarez 2014
Processing: "Thomas Randazzo 2014
Processing: "Anne Carter Blankenship 2014
Processing: "Dylan Lam 2014
Processing: "Shalaka Laxman 2014
Processing: "Kearby Chen 2009
Processing: "James Selph 2014
Processing: "David McGinley 2011
Processing: "Samraat Goel 2014
Processing: "Alexande

In [ ]:
def try_mcintyre(driver, first_name, last_name_school):
    driver.find_element_by_id('advanced-search').click()
    time.sleep(random.gauss(1.5, .5))
    
    driver.find_element_by_id('advs-firstName').clear()
    driver.find_element_by_id('advs-lastName').clear()
    driver.find_element_by_id('advs-school').clear()
    
    driver.find_element_by_id('advs-firstName').send_keys(first_name)
    driver.find_element_by_id('advs-lastName').send_keys(last_name)
    driver.find_element_by_id('advs-school').send_keys(school)
    driver.find_element_by_class_name('submit-advs').click()
    time.sleep(random.gauss(4.2, .5))
    

In [95]:
driver.close()

In [97]:
with open('results.csv', 'w') as output:
    scraper_writer = csv.writer(output)
    scraper_writer.writerow(['Input First','Input Last','Input Year','No. Search Results','Search Results Name',
                             'Search Results Work Summary', 'Current Position','Current Company','Current Location',
                             'Education Experience','Work Experience','Name Match','School Match','Year Match'])
    for row in scraped_info:
        scraper_writer.writerow(row)

In [44]:
testname = 'Kevin HsuPremium Badge1st'

In [41]:
testname = testname.strip('1st')

In [42]:
testname

'Kevin HsuPremium Badge'

In [43]:
testname = testname.rstrip('Premium Badge')
testname

'Kevin H'

In [45]:
testname

'Kevin HsuPremium Badge1st'

In [47]:
testname = testname.replace('1st', '')

In [48]:
testname

'Kevin HsuPremium Badge'

In [50]:
testname = testname.replace('Premium Badge', '')
testname

'Kevin Hsu'

In [59]:
work_experience

'Corporate Development\nVMware\nMay 2016 – Present (3 months)San Francisco Bay Area\nGrowth Equity Investor\nLevel Equity\nJune 2014 – May 2016 (2 years)New York, New York\nStrategy Consultant\nWells Fargo\n2013 – 2013 (less than a year)\nTrends Researcher\nPSFK\n2012 – 2012 (less than a year)\nResearch Assistant, Behavioral Economics Lab\nDuke University\n2009 – 2009 (less than a year)'

In [62]:
parsed_work_experience = work_experience.split('\n')

In [63]:
position = parsed_work_experience[0]
position

'Corporate Development'

In [65]:
company = parsed_work_experience[1]
company

'VMware'

In [66]:
dates = parsed_work_experience[2]
dates

'May 2016 – Present (3 months)San Francisco Bay Area'

In [67]:
location = dates
location

'San Francisco Bay Area'

In [86]:
num_results

'0'

In [6]:
def print_driver_url(driver):
    return "Driver url: {}".format(driver.current_url)

In [7]:
print_driver_url(driver=driver)

'Driver url: https://www.linkedin.com/'